In [1]:
from pulp import *
from alb_tools import *
import itertools


In [2]:
# pulpTestAll()

# Read Instance
Here we read instances and modify the tasks, precedence constraints and task times

In [3]:
instance_names = ['SALBP_benchmark/small data set_n=20/instance_n=20_1.alb','SALBP_benchmark/small data set_n=20/instance_n=20_2.alb' ]
test_instances = create_instance_pairs(instance_names)

In [4]:
!pwd

/Users/letshopethisworks2/Documents/Phd Paper material/MMABPWW


In [5]:
test_instances[0][0]

{'num_tasks': 20,
 'cycle_time': 1000,
 'order_strength': 0.268,
 'task_times': {'1': 142,
  '2': 34,
  '3': 140,
  '4': 214,
  '5': 121,
  '6': 279,
  '7': 50,
  '8': 282,
  '9': 129,
  '10': 175,
  '11': 97,
  '12': 132,
  '13': 107,
  '14': 132,
  '15': 69,
  '16': 169,
  '17': 73,
  '18': 231,
  '19': 120,
  '20': 186},
 'precedence_relations': [['1', '6'],
  ['2', '7'],
  ['4', '8'],
  ['5', '9'],
  ['6', '10'],
  ['7', '11'],
  ['8', '12'],
  ['10', '13'],
  ['11', '13'],
  ['12', '14'],
  ['12', '15'],
  ['13', '16'],
  ['13', '17'],
  ['13', '18'],
  ['14', '20'],
  ['15', '19']],
 'model_no': 0}

# Model Parameters

In [6]:
NO_EQUIPMENT = 4
NO_S = 2
C_EM = 300
NO_MODELS = 2
TAKT_TIME = 700 #cadence to respect
MAX_L = 3 #maximum number of workers at a station
#STATION_MODEL_PICTURES  = {}

S_E = [(s, e ) for s in range(NO_S) for e in range(NO_EQUIPMENT)]
S_L = [(s , l) for s in range(NO_S) for l in range(1, MAX_L)]

NO_CONFIG = NO_MODELS ** NO_S #Number of possible line pictures


all_tasks = list_all_tasks(test_instances[0])
c_se, r_oe = generate_equipment(NO_EQUIPMENT,NO_S, all_tasks)
stations = list(range(NO_S))
models = list(range(NO_MODELS))
model_pairs = list(itertools.combinations(models, 2))
omega = []
enumerate_universe('', omega, NO_MODELS, NO_S)
equipment = list(range(NO_EQUIPMENT))
workers = list(range(1, MAX_L+1))

print('cse',c_se)
print('r_oe', r_oe)

[[ True  True  True  True]
 [ True  True  True  True]
 [False  True  True  True]
 [ True  True  True  True]
 [ True  True  True  True]
 [ True  True  True  True]
 [ True  True  True  True]
 [ True  True  True False]
 [ True  True  True  True]
 [ True  True  True  True]
 [ True  True  True  True]
 [ True  True  True  True]
 [ True  True  True  True]
 [ True  True  True  True]
 [False  True  True  True]
 [ True  True  True  True]
 [ True  True  True  True]
 [ True  True  True  True]
 [ True  True  True  True]
 [ True  True  True  True]]
cse [[115 228 260 268]
 [265 217 229 105]]
r_oe [[ True  True  True  True]
 [ True  True  True  True]
 [False  True  True  True]
 [ True  True  True  True]
 [ True  True  True  True]
 [ True  True  True  True]
 [ True  True  True  True]
 [ True  True  True False]
 [ True  True  True  True]
 [ True  True  True  True]
 [ True  True  True  True]
 [ True  True  True  True]
 [ True  True  True  True]
 [ True  True  True  True]
 [False  True  True  True]
 [ Tru

In [7]:
all_tasks

['19',
 '17',
 '13',
 '6',
 '18',
 '14',
 '5',
 '20',
 '9',
 '8',
 '2',
 '16',
 '15',
 '1',
 '11',
 '10',
 '4',
 '3',
 '12',
 '7']

Create task time for tasks with multiple workers

In [8]:
for test_instance in test_instances:
    for model in test_instance:
        model[f'task_times_l'] = {}
        for i in workers:
            model[f'task_times_l'][i] = linear_reduction(model['task_times'], i)


In [9]:
def get_task_intersection(test_instance, model_1, model_2):
    return  set(test_instance[model_1]['task_times']).intersection(set(test_instance[model_2]['task_times']))
    


# Defining Fixed Task Assigment Model

Taken from Model-dependent task assignment in multi-manned mixed-model assembly lines with walking workers Hashemi-Petroodi 2022

## Decision Variables
NOTICE-- here I am using w as omega

Y -- # workers \
u_se -- # of pieces of equipment e at station s \
b_wsl --  1 if there are l workers in station s for picture w, 0 otherwise\
b_woisl -- 1 if there are l workers performing task o on model i at station s for picture w, 0 otherwise\
x_soi -- 1 if task o is performed on model i at station s, 0 otherwise

In [10]:
y = LpVariable("Y", lowBound=0, cat=LpInteger)
u_se = LpVariable.dicts('u_se',(stations,equipment), lowBound=0, cat= LpInteger)
b_wsl = LpVariable.dicts('b_wsl', (range(NO_CONFIG), stations, workers), lowBound=0, upBound=1, cat=LpBinary )
b_woisl = LpVariable.dicts('b_woisl',(range(NO_CONFIG), all_tasks, models, stations, workers),lowBound=0, upBound=1, cat=LpBinary)
x_soi = LpVariable.dicts('x_soi', (stations, all_tasks, models), lowBound=0, cat=LpBinary)

## DEFINING OBJECTIVE FUNCTION

Create variable for problem

In [11]:
prob = pulp.LpProblem("MALBP-W-FIX", LpMinimize)

The objective is to minimize the total cost of equipment and workers

In [12]:
test_instances[0][1]

{'num_tasks': 20,
 'cycle_time': 1000,
 'order_strength': 0.3,
 'task_times': {'1': 58,
  '2': 224,
  '3': 20,
  '4': 150,
  '5': 410,
  '6': 117,
  '7': 262,
  '8': 94,
  '9': 213,
  '10': 118,
  '11': 191,
  '12': 74,
  '13': 60,
  '14': 117,
  '15': 124,
  '16': 103,
  '17': 178,
  '18': 188,
  '19': 107,
  '20': 53},
 'precedence_relations': [['1', '13'],
  ['2', '5'],
  ['4', '6'],
  ['5', '10'],
  ['5', '11'],
  ['5', '12'],
  ['6', '7'],
  ['6', '8'],
  ['6', '9'],
  ['7', '13'],
  ['8', '13'],
  ['9', '13'],
  ['10', '16'],
  ['11', '14'],
  ['12', '15'],
  ['13', '17'],
  ['13', '18'],
  ['13', '19'],
  ['13', '20']],
 'model_no': 1,
 'task_times_l': {1: {'1': 58.0,
   '2': 224.0,
   '3': 20.0,
   '4': 150.0,
   '5': 410.0,
   '6': 117.0,
   '7': 262.0,
   '8': 94.0,
   '9': 213.0,
   '10': 118.0,
   '11': 191.0,
   '12': 74.0,
   '13': 60.0,
   '14': 117.0,
   '15': 124.0,
   '16': 103.0,
   '17': 178.0,
   '18': 188.0,
   '19': 107.0,
   '20': 53.0},
  2: {'1': 29.0,
   '2':

In [13]:
prob += ( lpSum([C_EM * y] + [c_se[s][e]* u_se[s][e] for (s,e) in S_E])
, 'TOTAL_COST') 

## Defining Constraints

In [14]:
def add_constraints_for_fixed_assignment(prob):
    for w_in, w in enumerate(omega):
        prob += pulp.lpSum([l * b_wsl[w_in][s][l] for s in stations for l in workers])<= y, ("tot Workers Pic" + str(w) ) #Constraint 2 in MILP_fix
        for s_in, s in enumerate(stations):
            prob += pulp.lpSum([b_wsl[w_in][s_in][l] for l in workers])==1, (f'workers_at_station_{s}_pic_{w}') #Constraint 3 in MILP_fix
            model_at_s = omega[w_in][s_in]
            tasks = test_instances[0][int(model_at_s)]['task_times_l']
            prob += pulp.lpSum([tasks[l][task]*b_woisl[w_in][task][int(model_at_s)][s_in][l] for l in workers for task in tasks[l].keys() ]) <= TAKT_TIME, (f'C for picture {w}, station {s}')#const. 9 
            for l in workers:
                for i in models:
                    for o in test_instances[0][i]['task_times']:
                        prob += l * b_woisl[w_in][o][i][s][l] <= pulp.lpSum([r_oe[int(o)-1][e] * u_se[s_in][e] for e in equipment]), (f' e that can cover woisl {w} {o} {i} {s} {l} ')#const. 10
        
    for model in models:
        for task in test_instances[0][model]['task_times']:
                prob+= pulp.lpSum([x_soi[s][task][model] for s in stations]) == 1, (f'task_{task}_assignment_model_{model}') #constraint 4
        for (pred, suc) in test_instances[0][model]['precedence_relations']:

            prob += pulp.lpSum([ (s+1)  * x_soi[s][pred][i] for s in stations]) <=  pulp.lpSum([ (s+1)  * x_soi[s][suc][i] for s in stations]), (f'task{pred} before task{suc} for model{model}') #const. 11
        
    for m1, m2 in model_pairs:
        shared_tasks = get_task_intersection(test_instances[0], m1, m2)
        for task in shared_tasks:
            for s in stations:
                prob += x_soi[s][task][m1] == x_soi[s][task][m2], (f'task {task} for models {m1, m2} at s {s}')#constraint 5

    for l in workers:
        for w_in, w in enumerate(omega):
            for i in models:
                for o in test_instances[0][i]['task_times']:
                    for s_in, station_picture in enumerate(w):
                        if str(model) == station_picture:
                            prob += b_woisl[w_in][o][i][s_in][l] <= x_soi[s_in][o][i], (f'{l} workers task{o} s{s_in} pic {w_in} mod {i} x_soi on') #constraint 6
                            prob += b_woisl[w_in][o][i][s_in][l] <= b_wsl[w_in][s_in][l],(f'{l} workers task{o} s{s_in} pic {w_in} mod {i} b_wsl on') #constraint 7
                            prob += b_woisl[w_in][o][i][s_in][l] >= b_wsl[w_in][s_in][l] + x_soi[s_in][o][i] - 1, (f'{l} workers task {o} s{s_in} pic {w_in} mod {i} b_wsl x_soi coord') #constraint 8
    return prob

add_constraints_for_fixed_assignment(prob)
                            

{'19', '17', '13', '6', '18', '14', '5', '20', '9', '8', '2', '16', '15', '1', '11', '10', '4', '3', '12', '7'}


MALBP-W-FIX:
MINIMIZE
300*Y + 115*u_se_0_0 + 228*u_se_0_1 + 260*u_se_0_2 + 268*u_se_0_3 + 265*u_se_1_0 + 217*u_se_1_1 + 229*u_se_1_2 + 105*u_se_1_3 + 0
SUBJECT TO
tot_Workers_Pic_'0',_'0'_: - Y + b_wsl_0_0_1 + 2 b_wsl_0_0_2 + 3 b_wsl_0_0_3
 + b_wsl_0_1_1 + 2 b_wsl_0_1_2 + 3 b_wsl_0_1_3 <= 0

workers_at_station_0_pic__'0',_'0'_: b_wsl_0_0_1 + b_wsl_0_0_2 + b_wsl_0_0_3
 = 1

C_for_picture__'0',_'0'_,_station_0: 175 b_woisl_0_10_0_0_1
 + 87.5 b_woisl_0_10_0_0_2 + 58.3333333333 b_woisl_0_10_0_0_3
 + 97 b_woisl_0_11_0_0_1 + 48.5 b_woisl_0_11_0_0_2
 + 32.3333333333 b_woisl_0_11_0_0_3 + 132 b_woisl_0_12_0_0_1
 + 66 b_woisl_0_12_0_0_2 + 44 b_woisl_0_12_0_0_3 + 107 b_woisl_0_13_0_0_1
 + 53.5 b_woisl_0_13_0_0_2 + 35.6666666667 b_woisl_0_13_0_0_3
 + 132 b_woisl_0_14_0_0_1 + 66 b_woisl_0_14_0_0_2 + 44 b_woisl_0_14_0_0_3
 + 69 b_woisl_0_15_0_0_1 + 34.5 b_woisl_0_15_0_0_2 + 23 b_woisl_0_15_0_0_3
 + 169 b_woisl_0_16_0_0_1 + 84.5 b_woisl_0_16_0_0_2
 + 56.3333333333 b_woisl_0_16_0_0_3 + 73 b_woisl_0_17

In [15]:
b_woisl

{0: {'19': {0: {0: {1: b_woisl_0_19_0_0_1,
     2: b_woisl_0_19_0_0_2,
     3: b_woisl_0_19_0_0_3},
    1: {1: b_woisl_0_19_0_1_1, 2: b_woisl_0_19_0_1_2, 3: b_woisl_0_19_0_1_3}},
   1: {0: {1: b_woisl_0_19_1_0_1,
     2: b_woisl_0_19_1_0_2,
     3: b_woisl_0_19_1_0_3},
    1: {1: b_woisl_0_19_1_1_1, 2: b_woisl_0_19_1_1_2, 3: b_woisl_0_19_1_1_3}}},
  '17': {0: {0: {1: b_woisl_0_17_0_0_1,
     2: b_woisl_0_17_0_0_2,
     3: b_woisl_0_17_0_0_3},
    1: {1: b_woisl_0_17_0_1_1, 2: b_woisl_0_17_0_1_2, 3: b_woisl_0_17_0_1_3}},
   1: {0: {1: b_woisl_0_17_1_0_1,
     2: b_woisl_0_17_1_0_2,
     3: b_woisl_0_17_1_0_3},
    1: {1: b_woisl_0_17_1_1_1, 2: b_woisl_0_17_1_1_2, 3: b_woisl_0_17_1_1_3}}},
  '13': {0: {0: {1: b_woisl_0_13_0_0_1,
     2: b_woisl_0_13_0_0_2,
     3: b_woisl_0_13_0_0_3},
    1: {1: b_woisl_0_13_0_1_1, 2: b_woisl_0_13_0_1_2, 3: b_woisl_0_13_0_1_3}},
   1: {0: {1: b_woisl_0_13_1_0_1,
     2: b_woisl_0_13_1_0_2,
     3: b_woisl_0_13_1_0_3},
    1: {1: b_woisl_0_13_1_1_1, 2: b_

In [16]:
path_to_cplex = '/Applications/CPLEX_Studio_Community2211/cplex/bin/x86-64_osx/cplex'
# solver = CPLEX_CMD(path=path_to_cplex, msg=True)
#solver = getSolver('CPLEX_PY')
#prob.numVariables()
prob.solve(solver=XPRESS_PY())


Using the license file found in your Xpress installation. If you want to use this license and no longer want to see this message, use the following code before using the xpress module:
  xpress.init('/Applications/FICO Xpress/xpressmp/bin/xpauth.xpr')
FICO Xpress v9.0.3, Community, solve started 10:52:22, May 25, 2023
Heap usage: 1550KB (peak 1550KB, 1022KB system)
Minimizing MILP noname using up to 8 threads and up to 8192MB memory, with these control settings:
OUTPUTLOG = 1
Original problem has:
      2535 rows         1073 cols         8848 elements      1073 entities
Presolved problem has:
      1879 rows          497 cols         5293 elements       497 entities
LP relaxation tightened
Presolve finished in 0 seconds
Heap usage: 2042KB (peak 3082KB, 1022KB system)

Coefficient range                    original                 solved        
  Coefficients   [min,max] : [ 1.00e+00,  4.10e+02] / [ 2.60e-02,  1.60e+00]
  RHS and bounds [min,max] : [ 1.00e+00,  7.00e+02] / [ 1.00e+00, 

1

In [17]:
prob

MALBP-W-FIX:
MINIMIZE
300*Y + 115*u_se_0_0 + 228*u_se_0_1 + 260*u_se_0_2 + 268*u_se_0_3 + 265*u_se_1_0 + 217*u_se_1_1 + 229*u_se_1_2 + 105*u_se_1_3 + 0
SUBJECT TO
tot_Workers_Pic_'0',_'0'_: - Y + b_wsl_0_0_1 + 2 b_wsl_0_0_2 + 3 b_wsl_0_0_3
 + b_wsl_0_1_1 + 2 b_wsl_0_1_2 + 3 b_wsl_0_1_3 <= 0

workers_at_station_0_pic__'0',_'0'_: b_wsl_0_0_1 + b_wsl_0_0_2 + b_wsl_0_0_3
 = 1

C_for_picture__'0',_'0'_,_station_0: 175 b_woisl_0_10_0_0_1
 + 87.5 b_woisl_0_10_0_0_2 + 58.3333333333 b_woisl_0_10_0_0_3
 + 97 b_woisl_0_11_0_0_1 + 48.5 b_woisl_0_11_0_0_2
 + 32.3333333333 b_woisl_0_11_0_0_3 + 132 b_woisl_0_12_0_0_1
 + 66 b_woisl_0_12_0_0_2 + 44 b_woisl_0_12_0_0_3 + 107 b_woisl_0_13_0_0_1
 + 53.5 b_woisl_0_13_0_0_2 + 35.6666666667 b_woisl_0_13_0_0_3
 + 132 b_woisl_0_14_0_0_1 + 66 b_woisl_0_14_0_0_2 + 44 b_woisl_0_14_0_0_3
 + 69 b_woisl_0_15_0_0_1 + 34.5 b_woisl_0_15_0_0_2 + 23 b_woisl_0_15_0_0_3
 + 169 b_woisl_0_16_0_0_1 + 84.5 b_woisl_0_16_0_0_2
 + 56.3333333333 b_woisl_0_16_0_0_3 + 73 b_woisl_0_17

In [18]:
varsdict = {}
for v in prob.variables():
    if v.varValue > 0:
        print(v.name, v.varValue)
    

Y 5.0
b_woisl_0_10_1_0_1 1.0
b_woisl_0_10_1_1_1 1.0
b_woisl_0_11_1_0_1 1.0
b_woisl_0_11_1_1_1 1.0
b_woisl_0_12_1_0_1 1.0
b_woisl_0_12_1_1_1 1.0
b_woisl_0_13_1_0_1 1.0
b_woisl_0_13_1_1_1 1.0
b_woisl_0_14_1_0_1 1.0
b_woisl_0_14_1_1_1 1.0
b_woisl_0_15_1_1_1 1.0
b_woisl_0_16_1_0_1 1.0
b_woisl_0_16_1_1_1 1.0
b_woisl_0_17_1_0_1 1.0
b_woisl_0_17_1_1_1 1.0
b_woisl_0_18_1_0_1 1.0
b_woisl_0_18_1_1_1 1.0
b_woisl_0_19_1_0_1 1.0
b_woisl_0_19_1_1_1 1.0
b_woisl_0_1_1_0_1 1.0
b_woisl_0_1_1_1_1 1.0
b_woisl_0_20_1_0_1 1.0
b_woisl_0_20_1_1_1 1.0
b_woisl_0_2_1_0_1 1.0
b_woisl_0_2_1_1_1 1.0
b_woisl_0_3_1_1_1 1.0
b_woisl_0_4_1_0_1 1.0
b_woisl_0_4_1_1_1 1.0
b_woisl_0_5_1_0_1 1.0
b_woisl_0_5_1_1_1 1.0
b_woisl_0_6_1_0_1 1.0
b_woisl_0_6_1_1_1 1.0
b_woisl_0_7_1_0_1 1.0
b_woisl_0_7_1_1_1 1.0
b_woisl_0_8_1_0_1 1.0
b_woisl_0_9_1_0_1 1.0
b_woisl_0_9_1_1_1 1.0
b_woisl_1_10_0_1_3 1.0
b_woisl_1_10_1_0_1 1.0
b_woisl_1_10_1_1_3 1.0
b_woisl_1_11_0_1_3 1.0
b_woisl_1_11_1_0_1 1.0
b_woisl_1_11_1_1_3 1.0
b_woisl_1_12_1_0_1 1.